# Data Import Script
This script pulls in all relevant data for a session in my PhD experiment in raw form; it will process the data and save it to pickles for use.

In [1]:
import os
import cv2
import time
import pandas as pd
import numpy as np
import seaborn as sns
import humanize
import re
import glob
import math
import datetime
import pickle
from IPython.display import display, clear_output, HTML
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import seaborn as sns
from csv import reader
from scipy.stats import beta
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R
from pympler import asizeof
from scipy import interpolate
from ahrs.filters import EKF
from ahrs import QuaternionArray
#%matplotlib inline
%matplotlib widget


TIMEZONE_OFFSET = 4

## E4 Data

by using `df = load_session_data('Data', 'P1', 'sess1')` this script will return a df with three columns: timestamp (ns), datatype (ACC, BVP, EDA, HR, IBI,or TEMP), and data (a list of values; typically a list with one value, but for acc we have three.)

This is one large df with all of the E4 data for that session.  Nicely for us, it is effectively 'done'.

To get dfs for each type of data separately, we can run `df_acc = extract_datatype(df, 'ACC')`.

What to do with the data here?  

#### ACC (32 Hz, m/s^2)

Accelerometer data comes in at 1/64 Gs (ints) and needs to be transformed by 1/64 g (that's equivalent to '1'; 128 should give us 2G, -128 should give us -2G).  We then multiply by 9.8m/s^2 to get a similar value to our accelerometer on the glasses.

For accelerometer data at the wrist, we might want to do a few things: (1) look over a time window and make a judgement about wrist movement-- is it a lot, or a little?  We may want to use that movement as a signal for how much we should trust the other sensors, which are contact based and can be pretty corrupted by motion artifacts.  

We can transform these three axes of motion into a vector, and see how different it is from an estimated gravity vector (resting, some stillness assumptions, etc).  Wrist stillness *may* tell us something about focus state, but probably not too much directly unless someone was engaged passively in something.  Perhaps the most interesting thing we could draw from it are patterns in the data, which summary statistics would destroy.  We can do a self-similarity matrix-style approach to build out some statistics on patterns in the accelerometer data

#### BVP (64 Hz, unitless reflectance ratio)

Raw PPG signal.  We may not need to look at this really-- our preference would be to derive HR and HRV/IBI from it anyway.  Maybe there is some interesting stuff in the raw signal, but that's out of scope.  Whats probably useful (if anything) is looking to see if the raw data looks trustworthy, and to check if we trust empatica's derivation of HR and IBI.  By running our own algorithm on the raw data, we might have a sense of confidence in our results.  We can also take into consideration the Accelerometer data once it's been transformed to a simple 'magnitude of motion' measure. 

#### HR (1 Hz, BPM)

A derived signal from the BVP, this is grabbing the anticipated HR.  We may want to assess the quality of BVP and their HR algorithm on our own to see how well they're doing; we may also want to throw away data when motion is an issue.  This derived signal lends itself easily to summary stats/time slices.

#### IBI (No Hz-- get them after each detected heartbeat-- so likely between 5 and 25 Hz, milliseconds)

This is a processed signal, we can probably just do some summary stats on it over various windows.  Once again it relies on the BVP raw signal.

#### EDA (4 Hz, uSiemens)

This is one of the signals that needs the most processing work; we typically do quite a bit of cleanup, and split out SCR (transients) from SCL (low-frequency content).  10% of users are estimated to be non-responders.  This one will require a lot of cleaning.

#### TEMP (4 Hz, degrees C)

No cleaning necessary for skin temperature.  We'll use it in concert with the other temperature values we've been collecting.




In [2]:
def generate_timestamps(length, start_time, sample_rate):
    time_offset = np.arange(length) / sample_rate
    timestamps = pd.to_datetime(start_time, unit='s', origin='unix') + pd.to_timedelta(time_offset, unit='s') - datetime.timedelta(hours=TIMEZONE_OFFSET)
    return timestamps


def process_file(file_path, file_name):
    with open(file_path, 'r') as file:
        # only take the first timestamp
        start_time = float(file.readline().split(',')[0].strip())
        df = pd.DataFrame()
        if 'IBI' in file_name:
            data = pd.read_csv(file, header=None, names=['time', 'ibi']).values.tolist()
            df['timestamp'] = pd.to_datetime([item[0] + start_time for item in data], unit='s', origin='unix')
        else:
            sample_rate = float(file.readline().split(',')[0].strip())
            data = pd.read_csv(file, header=None).values.tolist()
            df['timestamp'] = generate_timestamps(len(data), start_time, sample_rate)
            
            if 'ACC' in file_name:
                #accelerometer data is in 1/64 Gs.  Our glasses data is in m/s^2. Transform it to m/s^2
                data = [[(i/64)*9.8 for i in l] for l in data]
        df['datatype'] = file_name.split('.')[0]
        df['data'] = data
        return df

    
def load_session_data(folder, participant_id, session_id):
    data_dir = f'{folder}/{participant_id}/{participant_id}_{session_id}'
    data_frames = []

    expected_files = ['ACC.csv', 'BVP.csv', 'EDA.csv', 'HR.csv', 'IBI.csv', 'TEMP.csv']

    #load all files, save the output of 'process_file' into data_frames
    for subdir, dirs, files in os.walk(data_dir):
        for file in files:
            if file in expected_files:
                file_path = subdir + os.sep + file
                df = process_file(file_path, file)
                data_frames.append(df)
    
    #turn data_frames into df
    final_df = pd.concat(data_frames)
    final_df.set_index('timestamp', inplace=True)
    final_df.sort_index(inplace=True)
    
    return final_df


def extract_datatype(df, datatype):
    # Filter the dataframe for the given datatype
    df_filtered = df[df['datatype'] == datatype].copy()
    
    # Determine the number of columns based on the length of the 'data' list in the first row
    num_cols = len(df_filtered['data'].iloc[0])
    
    # Create column names
    if datatype == 'ACC':
        col_names = ['x','y','z']
    else:
        col_names = ['d' + str(i + 1) for i in range(num_cols)]
    
    # Split the 'data' list into multiple columns
    df_split = pd.DataFrame(df_filtered['data'].to_list(), columns=col_names)
    
    # Add timestamp back to the dataframe
    df_split.index = df_filtered.index
    
    return df_split

def plot_df(df, title):
    fig, ax = plt.subplots(figsize=(15, 3))

    # Assuming df has timestamp as index
    for column in df.columns:
        if 'tick' not in column:
            sns.lineplot(data=df, x=df.index, y=column, ax=ax, lw=1.5)
        
    # Format the x axis
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

    # Set grid
    ax.grid(True, which="both", color='gray', linewidth=0.25)

    # Set title and labels
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Time', fontsize=11)
    ax.set_ylabel('Value', fontsize=11)

    # Set legend
    #ax.legend(loc='upper left', fontsize=14)

    # Set tick parameters
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    # Improve date tick display
    fig.autofmt_xdate()

    plt.show()

## Glasses and Watch Data

We have more work to do on the glasses data, because it doesnt come with the preprocessing completed for us.  We have:

(1) Accelerometer Data.  We can use similar accelerometer code as above.

(2) Gyro Data.  We can combine with Accelerometer in a Kalman filter to get a sense of head orientation.  Potentially very interesting.

(3) Nose and Temple Temp and Ambient Temp/Humidity data.  These come in at different rates and have different calibrations, but once applied this is straightforward to work with (we just have to do the processing).

(4) Blink data.  Potentially a lot of sophisticated stuff to pull out here, but we can just start with a simple script that has some basic hueristics about blink duration and frequency content built into it.

(5) Ambient Light data.  No real hard stuff here, just gotta calibrate correctly.

In [3]:
def get_size_mb(obj):
    size_bytes = asizeof.asizeof(obj)
    size_mb = size_bytes / (1024 * 1024)  # Convert bytes to megabytes
    return size_mb

def interpolate_df(df, start_timestamp, end_timestamp, new_freq):
    # Calculate the total time duration of the data in seconds
    total_time = (end_timestamp-start_timestamp).total_seconds()
    
    # Create new timestamp index at the desired frequency
    new_index = pd.date_range(start=start_timestamp, periods=int(total_time * new_freq), freq=f'{1/new_freq*1000}ms')

    # Initialize new DataFrame with the new index
    df_new = pd.DataFrame(index=new_index)

    # For each column in the original DataFrame, interpolate to the new frequency
    for col in df.columns:
        # Create spline interpolator for this column
        interpolator = interpolate.interp1d(df.index.values.astype(float), df[col].values, kind='cubic')

        # Interpolate this column at the new timestamps
        df_new[col] = interpolator(new_index.values.astype(float))

    return df_new

def load_session_excel(folder, participant_id, session_id):
    
    # grab all filepaths in there, in order.
    data_dir = f'{folder}/{participant_id}/{participant_id}_{session_id}'
    filepaths = sorted(glob.glob( data_dir + '/*.csv'))
    
    session_data = []
    
    # process all of them
    for file in filepaths:
        
        print('\n\nProcessing ' + file)
        # pull out timestamp and session type
        pattern = r".*/(\d{6})_(\d{6})_p\d+_(.*).csv"
        match = re.search(pattern, file)
        
        date_str = match.group(1)
        time_str = match.group(2)
        session_type = match.group(3)
        
        # Convert the date and time to a pandas.Timestamp object
        timestamp = pd.to_datetime(date_str + time_str, format='%m%d%y%H%M%S')
        
        logs, blinks, acc, gyro, thermal, ui, wtemp, wlux = [], [], [], [], [], [], [], []

        with open(file, 'r') as read_obj:
            csv_reader = reader(read_obj)
            for row in csv_reader:
                try:
                    if   (row[0]=='l'):
                        logs.append(row[1:])
                    elif (row[0]=='g'):
                        if   (row[2]=='b'):
                            blinks.append(row[1:])
                        elif (row[2]=='t'):
                            thermal.append(row[1:])
                        elif (row[2]=='a'):
                            acc.append(row[1:])
                        elif (row[2]=='g'):
                            gyro.append(row[1:])
                    elif (row[0]=='u'):
                        ui.append(row[1:])
                    elif (row[0]=='w'):
                        if(row[3] == 'TX_TEMP_HUMD'):
                            wtemp.append(row[1:])
                        elif(row[3] == 'TX_LUX_WHITELUX'):
                            wlux.append(row[1:])
                except Exception as e:
                    print(str(e) + ': ' + str(row))

        print('got raw data.')
        
        session = {
            'user': participant_id,
            'session': session_id,
            'filepath': file,
            'session_type': session_type,
            'timestamp': timestamp,
            'logs': logs,
            'ui': ui
        }
        
        print('\tthermal processing...')
        try:
            check_raw_packets(thermal, timeout_ms=4200)
            thermal_df = pd.DataFrame()
            for t in thermal:
                thermal_df = pd.concat([thermal_df, parseThermalRow(t)], axis=0)    
            thermal_df.reset_index(inplace=True)
            thermal_df = reindex_to_timestamp(thermal_df)
            thermal_df['nose_tp_rolling'] = thermal_df['nose_tp'].rolling(7).mean()
            thermal_df['temple_tp_rolling'] = thermal_df['temple_tp'].rolling(2).mean()
            thermal_df['nose_thermistor_rolling'] = thermal_df['nose_thermistor'].rolling(40).mean()
            thermal_df['temple_thermistor_rolling'] = thermal_df['temple_thermistor'].rolling(40).mean()
            thermal_df['nose_temp_C'] = thermal_df.apply(lambda x: convert_nose_temp(x['nose_tp_rolling'], x['nose_thermistor_rolling']), axis=1)
            thermal_df['temple_temp_C'] = thermal_df.apply(lambda x: convert_temple_temp(x['temple_tp_rolling'], x['temple_thermistor_rolling']), axis=1)
            thermal_df['differential_C'] = thermal_df['nose_temp_C'] - thermal_df['temple_temp_C']
            thermal_df['differential_C_rolling'] = thermal_df['differential_C'].rolling(15).mean() 
            session['thermal']= thermal_df
        except Exception as e:
            print('FAILED TO PROCESS DATA!!!! ' + str(e))
            
        print('\tgyro processing...')
        try:
            check_raw_packets(gyro, timeout_ms=900)
            gyro_df = parse_xyz_data(gyro)
            gyro_df.reset_index(inplace=True)
            gyro_df = reindex_to_timestamp(gyro_df)
            gyro_df = gyro_df[gyro_df['x'] < 10000]
            gyro_df = gyro_df[gyro_df['y'] < 10000]
            gyro_df = gyro_df[gyro_df['z'] < 10000]

            #convert int16s to floats with q=9.
            #this gives us rad/s (angular velocity), we expect values in the 0.3-3 range.
            #Gyro set to updates every 10ms, or 100Hz
            gyro_df['x'] = gyro_df['x'].astype(float) / 512
            gyro_df['y'] = gyro_df['y'].astype(float) / 512
            gyro_df['z'] = gyro_df['z'].astype(float) / 512
            session['gyro']= gyro_df
        except Exception as e:
            print('FAILED TO PROCESS DATA!!!! ' + str(e))
       
        print('\taccelerometer processing...')
        try:
            check_raw_packets(acc, timeout_ms=1350)
            acc_df = parse_xyz_data(acc)
            acc_df.reset_index(inplace=True) 
            acc_df = reindex_to_timestamp(acc_df)
            acc_df = acc_df[acc_df['x'] < 10000]
            acc_df = acc_df[acc_df['y'] < 10000]
            acc_df = acc_df[acc_df['z'] < 10000]

            #convert int16s to floats with q=8.
            #this gives us data in m/s^2, so we expect ~9.8 when at rest
            #accelerometer set for every 20ms, or 50Hz
            acc_df['x'] = acc_df['x'].astype(float) / 256
            acc_df['y'] = acc_df['y'].astype(float) / 256
            acc_df['z'] = acc_df['z'].astype(float) / 256
            session['acc']= acc_df
        except Exception as e:
            print('FAILED TO PROCESS DATA!!!! ' + str(e))
            
        print('\tquaternion extended kalman filter application...')
        try:
            start_timestamp = max(acc_df.index[0], gyro_df.index[0])
            end_timestamp = min(acc_df.index[-1], gyro_df.index[-1])

            acc_inter = interpolate_df(acc_df, start_timestamp, end_timestamp, 100)
            gyro_inter = interpolate_df(gyro_df, start_timestamp, end_timestamp, 100)

            ekf = EKF(gyr=gyro_inter[['x','y','z']].values, acc=acc_inter[['x','y','z']].values, frequency=100.0)
            qs = QuaternionArray(ekf.Q)
            qs.remove_jumps()
            qs_df = pd.DataFrame({'w': qs[:,0], 'i':qs[:,1], 'j':qs[:,2], 'k':qs[:,3]}, index=acc_inter.index)
            session['quaternions']= qs_df
        except Exception as e:
            print('FAILED TO PROCESS DATA!!!! ' + str(e))
            
        print('\tblink processing...')
        try:
            check_raw_packets(blinks, timeout_ms=500)
            blinks_df = parse_blink_rows(blinks)
            blinks_df.reset_index(inplace=True)
            blinks_df = reindex_to_timestamp(blinks_df)
            session['blinks']= blinks_df
        except Exception as e:
            print('FAILED TO PROCESS DATA!!!! ' + str(e))
            
        print('\twatch processing...')
        try:
            session['watch_temp']= processWatchTemp(wtemp)
            session['watch_lux'] = processWatchLux(wlux)     
        except Exception as e:
            print('FAILED TO PROCESS DATA!!!! ' + str(e))
        
        session_data.append(session)
    
    print(f"The size of the loaded data is {get_size_mb(session_data)} MB")
    return session_data


def processWatchTemp(watch_data):
    '''Apply simple calibration to temp data based on comparison with cheap temp/humd gauge from amazon.
    Also, process the timestamp and turn it into a df'''
    
    to_df = []    
    
    for item in watch_data:
        to_df.append({
            'timestamp':pd.to_datetime(item[1]),
            'temp': 21.2 + (6.7/5.4) * (float(item[3])-24),
            'humidity': float(item[4])
        })
    
    df = pd.DataFrame(to_df)

    try:
        df['timestamp'] = df['timestamp'].dt.tz_localize(None)
        df.set_index('timestamp', inplace=True)
    except:
        print('\t\tno entries')

    return df

def processWatchLux(watch_data):
    
    to_df = []    
    for item in watch_data:
        to_df.append({
            'timestamp':pd.to_datetime(item[1]),
            'lux': float(item[3]),
            'whitelux': float(item[4])
        })
    
    df = pd.DataFrame(to_df)
    try:
        df['timestamp'] = df['timestamp'].dt.tz_localize(None)
        df.set_index('timestamp', inplace=True)
    except:
        print('\t\tno entries')
    return df

def parse_blink_rows(blinks):
    blink_rows = []
    
    for b in blinks:
        packetLength = int(b[6])
        blink_df_row = pd.DataFrame(data=[b[0:12] for i in range(packetLength)], 
                                    columns=['serverTS', 'packetTypeLetter', 'packetType', 'packetNum', 
                                             'msFromStart', 'epoch', 'packetSize', 'res0', 'res1', 'res2','res3','res4']) 
        
        
        blink_df_row["data"] = pd.Series([int(v) for v in b[13:]], dtype=np.int32)
        blinkSampleRate = 2000 # 2kHz
        blinkSamplePeriod_ms = 1000 * (1/blinkSampleRate) # 0.5 ms
        blink_df_row["tick_ms"] = blink_df_row['msFromStart'].astype(int)
        blink_df_row["tick_ms"] /= 2
        blink_df_row["tick_ms"] -= (packetLength - np.array(range(1, packetLength+1))) * blinkSamplePeriod_ms
        blink_df_row["saturated"] = blink_df_row["res0"].astype(int)
        blink_rows.append(blink_df_row)   
     
    return pd.concat(blink_rows)
 
    
def parse_xyz_data(data):
    def parseXYZRow(p):
        packetLength = 25
        base_data = p[0:12]
        other_data = np.array(p[13:]).astype('int32').reshape(-1,5)
        return [(base_data + list(row)) for row in other_data]

    data_out = []
    for d in data:
        data_out.extend(parseXYZRow(d))

    df = pd.DataFrame(data_out, columns=['serverTS', 'packetTypeLetter', 'packetType', 'packetNum', 'msFromStart', 'epoch', 'packetSize', 'res0', 'res1', 'res2','res3','res4', 'x','y','z','imu_tick_ms','tick_ms'])
    
    df["tick_ms"] = df["tick_ms"]/2
    
    return df


def parseThermalRow(t):
    tempSamplePeriod_ms = 100
    numInnerPackets = 4
    rowsInInnerPacket = 5
    # each row has 128 values; 4x 32 value 'packets'; 
    # each 32 has 5 repetitions of 6 values of data + 2 (tick,epoch)
    # these 6 values are interleaved; i.e. the first three are temple numbers, and the first
    # 15 sets of three are temple vals; the second 15 sets of 3 are the corresponding nose vals.
    # to get a timestamped packet, we need [0,1,2] matched with [15,16,17] and so on within each
    # 32 value 'packet'.  
    
    df_row = pd.DataFrame(data=[t[0:12] for i in range(numInnerPackets*rowsInInnerPacket)], columns=['serverTS', 'packetTypeLetter', 'packetType', 'packetNum', 'msFromStart', 'epoch', 'packetSize', 'res0', 'res1', 'res2','res3','res4']) 
    
    payload = np.array(t[13:]).astype(np.int32).reshape(numInnerPackets, -1)
    
    def flatten(t): return [item for sublist in t for item in sublist]
    reindex = flatten([[*range(i*3,i*3+3)] + [*range(15+i*3, 18+i*3)] + [-2,-1] for i in range(rowsInInnerPacket)])
    proper_payload = payload[:,reindex].reshape(-1,8)
    
    df_row = pd.concat([df_row, pd.DataFrame(data=proper_payload, columns=["temple_tp","temple_thermistor","secondary_temple_tick_ms","nose_tp","nose_thermistor","secondary_nose_tick_ms","tick_ms","epoch"])], axis=1)
    df_row['tick_ms'] -= tempSamplePeriod_ms * np.array([4,3,2,1,0] * numInnerPackets)
    
    return df_row

def convert_thermistor_to_K(raw_value):
    # STEP 1. Calc R_thermistor value given ADC_val
    R_divider = 100000
    ADC_max = 4095
    
    R_t = (R_divider * raw_value/ADC_max) / (1 - raw_value/ADC_max)
    
    # STEP 2. Convert R_t to temp
    R_0 = 100000
    T_0 = 298.15
    Beta = 3960
    R_inf = R_0 * math.exp(-Beta/T_0)
    
    Temp_K = Beta / np.log(R_t / R_inf)
    return Temp_K
    
def convert_thermopile_to_C(raw_val, T_ref_K, A, a_0, a_1):
    #STEP 1. Calc V_thermopile based on preamp gain and bias
    V_tp = (((raw_val / 4095) * 3.3) - (1.15+0.6084))/1000.0
    
    #STEP 2. Solve for Temperature!
    f_V_tp = (V_tp - a_0) + a_1 * (V_tp - a_0)**2
    T_obj = (T_ref_K**4 + f_V_tp/A)**0.25
    return (T_obj - 273.15)
    
def convert_nose_temp(raw_val, raw_thermistor):
    A   =  7.8e-10
    a_0 = -2.31e-01
    a_1 =  3.61e-03
    try:
        return convert_thermopile_to_C(raw_val, 
                                  convert_thermistor_to_K(raw_thermistor), 
                                  A, a_0, a_1)
    except:
        return np.nan
    
def convert_temple_temp(raw_val, raw_thermistor):
    A   =  4.21e-10
    a_0 = -3.62e-01
    a_1 =  8.31e-02
    try:
        return convert_thermopile_to_C(raw_val, 
                                  convert_thermistor_to_K(raw_thermistor), 
                                  A, a_0, a_1)
    except:
        return np.nan
    

def check_raw_packets(data, timeout_ms=500):
    last_seen = pd.to_datetime(data[0][0]) #0 = serverTimestamp
    last_tick = int(data[0][4]) #4 = packet_tick
    last_packet = int(data[0][3]) #3 = packetNum
    packets = set() 
    packets.add(last_packet)

    for i, d in enumerate(data[1:]):

        current_seen = pd.to_datetime(d[0])
        current_packet = int(d[3])
        current_tick = int(d[4])

        # Check if serverTimestamp is proceeding in order, which it should. 
        if current_seen < last_seen:
            print(f"\t\t-- at index {i+1}: Timestamp out of order. Current: {current_seen}, Previous: {last_seen}")

        # Check for duplicate packetNumbers by checking for the number in set packets.
        #if current_packet in packets:
        #    print(f"Error at index {i+1}: Duplicate packetNum detected. packetNum: {current_packet}")

        # Make sure packetNum is one larger than previous.
        if current_packet != last_packet + 1 and current_packet != 0:
            print(f"\t\tat index {i+1}: packetNum not sequentially increasing. Current: {current_packet}, Previous: {last_packet}")

        # Make sure tick has increased from last_tick.
        if current_tick <= last_tick:
            print(f"\t\tat index {i+1}: packet_tick not increasing. Current: {current_tick}, Previous: {last_tick}")

        # Check if the current tick is no more than 500ms greater than last tick.
        if current_tick - last_tick > timeout_ms:
            print(f"\t\tat index {i+1}: tick increment greater than timeout. Difference: {current_tick - last_tick}ms")

        # Update last_seen, last_tick, last_packet, packets
        last_seen = current_seen
        last_tick = current_tick
        last_packet = current_packet

        
        
def check_time_deltas(data):
    ''' feed in raw data after reading it into a list, this will just check
    server timestamps vs packet vs ticks and look at the differences elapsed 
    from packet to packet.  For debugging. '''
    prev_server = pd.to_datetime(data[0][0])
    prev_packet = int(data[0][3])
    prev_tick   = int(data[0][4])

    for d in data[1:200]:
        cs = pd.to_datetime(d[0])
        cp = int(d[3])
        ct = int(d[4])
        
        print(f'Differences: PACKET:{cp-prev_packet}\tSERVER:{humanize.precisedelta(cs-prev_server)}\tTICK:{(ct-prev_tick)/1000}')
        
        prev_server = cs
        prev_packet = cp
        prev_tick   = ct
        

def reindex_to_timestamp(df, column='tick_ms'):
    '''assumes a serverTS and tick_ms.  grabs first serverTS, assumes it marks 
    the tick_ms of the last row in the first packet, and then uses the tick_ms
    as the gold standard for all other timestamps'''
    
    df = df.copy()

    #make sure column is a datetime type
    df[column] = pd.to_datetime(df[column], unit='ms')
    
    # Set the first serverTS as the base timestamp
    base_timestamp = pd.to_datetime(df.loc[0, 'serverTS'])

    # Find the tick_ms of the last entry in the first packet by looking at the change in packetNum
    first_packet_num = df.loc[0, 'packetNum']
    zero_time_index = np.where(df['packetNum'].shift(-1) != first_packet_num)[0]
    
    if len(zero_time_index) > 0:
        zero_time = df.loc[zero_time_index[0], column]
    else:
        zero_time = df.loc[0, column]

    # Adjust tick_ms to get the correct offset
    df.loc[:, column] -= zero_time
    df.loc[:, column] = pd.to_timedelta(df[column], unit='ms')

    
    # Create new timestamp and set as index
    df['timestamp'] = base_timestamp + df[column]
    
    # Drop duplicates
    l = len(df)
    df.drop_duplicates(subset='timestamp', keep='first', inplace=True)
    print(f'\t\tdropped {l-len(df)} rows of data due to duplication.')
    
    # Drop data that's not within 10 sec of first and last server timestamp
    l = len(df)
    cutoff = pd.to_datetime(df.iloc[0]['serverTS']) - datetime.timedelta(seconds=10)
    df = df[df['timestamp'] > cutoff]
    cutoff = pd.to_datetime(df.iloc[-1]['serverTS']) + datetime.timedelta(seconds=10)
    df = df[df['timestamp'] < cutoff]
    print(f'\t\tdropped {l-len(df)} rows of data due to weird timestamps.')
    
    df['timestamp'] -= datetime.timedelta(hours=TIMEZONE_OFFSET)
    
    df.set_index('timestamp', inplace=True)
    
    print('\t\t-----------------------------')
    #print('\t\t-- reindexed timestamps based on first packet server timestamp and last tick_ms of first packet')
    #print('\t\t-- there is the possibility of drift with this technique; check that these values of the')
    #print('\t\t-- final packet are roughly in sync.')
    #print('\t\t-- First server time: ' + pd.to_datetime(df['serverTS'].iloc[0]).time().isoformat())
    #print('\t\t-- Last server time: ' + pd.to_datetime(df['serverTS'].iloc[-1]).time().isoformat())
    #print('\t\t-- First timestamp: ' + df.index[0].time().isoformat())
    #print('\t\t-- Last timestamp: ' + df.index[-1].time().isoformat())
    #print('\t\t-- Duration serverTS:  ' + humanize.precisedelta(pd.to_datetime(df['serverTS'].iloc[-1])-pd.to_datetime(df['serverTS'].iloc[0])))
    #print('\t\t-- Duration timestamp: ' + humanize.precisedelta(df.index[-1]-df.index[0]))
    print('\t\tDuration: ' + humanize.precisedelta(df.index[-1]-df.index[0]))
    print('\t\tError (server timestamp vs tick): ' + humanize.precisedelta(pd.to_datetime(df['serverTS'].iloc[-1]) - df.index[-1] - datetime.timedelta(hours=TIMEZONE_OFFSET)))
    print('\t\t% drift: ' + str(100.0*(pd.to_datetime(df['serverTS'].iloc[-1]) - df.index[-1] - datetime.timedelta(hours=TIMEZONE_OFFSET))/(df.index[-1]- df.index[0])))
     
    df.drop([column, 'serverTS', 'msFromStart','index','packetTypeLetter','packetType','packetNum','epoch','packetSize','res0','res1','res2','res3','res4'], axis=1, inplace=True)

    return df

In [4]:
def chop_df(df, taskbounds):
    """
    This function filters a dataframe based on the provided start and end times.
    The dataframe should have timestamps as the index.
    
    Args:
        df (pd.DataFrame): Input dataframe with timestamps as the index.
        taskbounds: Tuple of (start_time, end_time), where each is an (np.datetime64)
        
    Returns:
        pd.DataFrame: A new dataframe with only the data between the provided start and end times.
    """
    start_time, end_time = taskbounds
    
    start_time = pd.to_datetime(start_time).replace(tzinfo=None)
    end_time = pd.to_datetime(end_time).replace(tzinfo=None)

    # Filter the dataframe
    df_filtered = df[(df.index >= start_time) & (df.index <= end_time)]

    return df_filtered


#process E4 data into sessions
def update_sessions_with_E4_data(session_data, folder, user, session):
    
    #load E4 data
    df = load_session_data(folder, 'P' + str(user), 'sess' + str(session))
    
    df_names = ['ACC', 'IBI', 'TEMP', 'EDA', 'BVP', 'HR']
    
    #iterate over each data_type in the E4 session
    for df_type in df_names:
        
        #get just that data
        df_temp = extract_datatype(df, df_type)
        
        #now iterate over sessions in session_data
        for i, session in enumerate(session_data):
            
            try:
                #grab start/end timestamps; let's base it on blink data, which is sampled the fastest
                start_time = session['blinks'].index[0]
                end_time = session['blinks'].index[-1]

                #grab just the relevant section for this session of this datatype and append it into the session_data
                session_data[i]['E4_' + df_type] = chop_df(df_temp, (start_time, end_time))
            except Exception as e:
                print(f'Failed to add E4 {df_type} for session {session["session_type"]}')
                print(e)

## Behavioral and Survey Data

We need to process some data that will be useful for us to make estimates about what's going on here.  Below we'll start to group the data from the surveys by section:

These have specific ways to process.

- standard flow test data for each section (FSS, etc)
- standard wellbeing questionaire 
- standard TAS (absorption trait)

These do not:
- standard affect (affect model of flow) -- alertness and valence, reaction times, stress (interesting work to look at alertness vs. reaction time).
- free text answers about caffeine, life, stress, factors that matter that we're not measuring.
- enjoyment of tetris, enjoyment of games, self-perceived skill

Things that are going to get interesting to model and definitely don't have prior art for processing them:

- time perception data (durations, clock time, confidence)
- duration voluntarily continuing the task
- duration until noticing of light
- additional flow datapoints, including the drawn data.

In this section, we'll just organize the data.

In [9]:
#All combined

def get_user_entrance_responses(folder, user):
    '''grab the entrance survey results from google form csv'''
    
    df = pd.read_csv(folder + '/Flow_Entrance_Survey.csv')

    # Filter DataFrame to get row of specified user  
    user_row = df[df['Participant number (from the email you received):'] == user]
    
    # Ignore the timestamp and participant ID column
    user_responses = user_row.iloc[:, 2:].values.tolist()
    
    # user_responses is a list of lists, we flatten it to a single list
    user_responses = [item for sublist in user_responses for item in sublist]
    
    
    survey_data = {}
    
    #TAS : scale 1-5
    TAS_responses = user_responses[21:55]
    #BIT : scale 1-7
    BIT_responses = [v* 5 / 7 for v in user_responses[55:65]]
    del user_responses[21:65]
    #Flow Desc : Y/N
    FlowDesc_responses = user_responses[11:17]
    del user_responses[11:17]
    
    #CODE TAS
    #TAS just sum for likert 5 point version
    # based on prior work, avg ~80, std-dev ~18 (Glisky et al., 1991, Studies 2 and 3)
    survey_data['TAS'] = sum(TAS_responses)
    survey_data['TAS_stdevs'] = (sum(TAS_responses) - 80) / 18.0
    #CODE BIT
    BIT_codes = [
     'Support', 
     'Belonging',
     'Flow',
     'Accomplishment',
     'Self-efficacy',
     'Self-worth',
     'Meaning and Purpose',
     'Optimism',
     'Life Satisfaction',
     'Positive Affect'
    ]    
    strengths, weaknesses = [],[]

    for i, v in enumerate(BIT_responses):
        if v > 4.2:
            strengths.append(BIT_codes[i])
        elif v < 3.3:
            weaknesses.append(BIT_codes[i])

    BIT_mean = np.mean(BIT_responses)
    BIT_std = np.std(BIT_responses)
    print(f'BIT CODING:\t{BIT_mean:.2f} with std {BIT_std:.2f}', end='')

    if BIT_mean > 4.2:
            print(' -- DESIGNATED "STRONG"')
    elif BIT_std < 3.3:
            print(' -- DESIGNATED "AT RISK"')

    print(f'\t\t{len(strengths)} BIT STRENGTHS:\n\t\t {strengths}')
    print(f'\t\t{len(weaknesses)} BIT WEAKNESSES:\n\t\t {weaknesses}')

    survey_data['BIT_score'] = BIT_mean
    survey_data['BIT_std'] = BIT_std
    survey_data['BIT_strengths'] = strengths
    survey_data['BIT_weaknesses'] = weaknesses
    
    
    #CODE FQ
    s_score, d_score = 0, 0
    
    for r in FlowDesc_responses[:3]:
        if r == 'Yes':
            s_score +=1
    for r in FlowDesc_responses[3:]:
        if r == 'Yes':
            d_score +=1
    
    print(f'FLOW DESC:\tshallow: {s_score}\tdeep: {d_score}')
    survey_data['shallowFQ'] = s_score
    survey_data['deepFQ'] = d_score
    
    #remaining responses
    resp_keys = ['sex', 'job', 'typical_day', 'focus_best', 
     'vidgame_experience', 'vidgame_enjoy_1to7', 'vidgame_experience', 'vidgame_favtypes',
     'tetris_experience',  'tetris_enjoy_1to7',  'tetris_experience', 'flow_experiences',
     'flow_freq', 'flow_reliability', 'flow_recognition', 'flow_activity']
    
    for i, k in enumerate(resp_keys):
        survey_data[k] = user_responses[i]
        
    return survey_data


def get_user_exit_responses(folder, user):
    '''get user results for the final session, hand coded from their booklet into Final_PhD_Survey_Results.csv'''
    
    df = pd.read_csv(folder + '/Final_PhD_Survey_Results.csv')

    questions = list(df['Question'].values)
    user_responses = list(df['P' + str(user)].values)
    
    survey_data = {}
    
    #BIT : scale 1-7
    BIT_responses = [(int(v)+1)* 5 / 7 for v in user_responses[39:49]]
    del questions[39:49]
    del user_responses[39:49]
    #Flow Desc : Y/N
    FlowDesc_responses = user_responses[26:32]
    del questions[26:32]
    del user_responses[26:32]
    
    #CODE BIT
    BIT_codes = [
     'Support', 
     'Belonging',
     'Flow',
     'Accomplishment',
     'Self-efficacy',
     'Self-worth',
     'Meaning and Purpose',
     'Optimism',
     'Life Satisfaction',
     'Positive Affect'
    ]    
    strengths, weaknesses = [],[]

    for i, v in enumerate(BIT_responses):
        if v > 4.2:
            strengths.append(BIT_codes[i])
        elif v < 3.3:
            weaknesses.append(BIT_codes[i])

    BIT_mean = np.mean(BIT_responses)
    BIT_std = np.std(BIT_responses)
    print(f'BIT CODING:\t{BIT_mean:.2f} with std {BIT_std:.2f}', end='')

    if BIT_mean > 4.2:
            print(' -- DESIGNATED "STRONG"')
    elif BIT_std < 3.3:
            print(' -- DESIGNATED "AT RISK"')

    print(f'\t\t{len(strengths)} BIT STRENGTHS:\n\t\t {strengths}')
    print(f'\t\t{len(weaknesses)} BIT WEAKNESSES:\n\t\t {weaknesses}')

    survey_data['BIT_score'] = BIT_mean
    survey_data['BIT_std'] = BIT_std
    survey_data['BIT_strengths'] = strengths
    survey_data['BIT_weaknesses'] = weaknesses
    
    #CODE FQ
    s_score, d_score = 0, 0
    
    for r in FlowDesc_responses[:3]:
        if r == 'y':
            s_score +=1
    for r in FlowDesc_responses[3:]:
        if r == 'y':
            d_score +=1
    
    print(f'FLOW DESC:\tshallow: {s_score}\tdeep: {d_score}')
    survey_data['shallowFQ'] = s_score
    survey_data['deepFQ'] = d_score
    
    #remaining responses
    
    resp_keys = ['durGuess','durTimeConfidence','guessTime','guessTimeConfidence','timeExp',
    'focus','effort','deepest','flow','durFlow','durToFlow','percentFlow',
    'flowDesc','flowQ1','flowQ2','flowQ3','flowQ4','FSS1','FSS2','FSS3','FSS4',
    'FSS5','FSS6','FSS7','FSS8','FSS9','nowAlertness','nowStress','nowEmotion',
    'nowEmoIntensity','written_description','written_food_caff_exercise_sleep',
    'written_other_relevant','written_comments','written_homevlab','written_timechoice',
    'written_iphonepaper','written_observed','written_naturalwearables']
    #missing (obviously) timeAtGuess and timeAtDurGuess.  We'll use the last timestamp of the wearables for the time at guess.
    #we don't really care about the timeAtDurGuess (just use duration of flow activity-- we do care about timeAtGuess.

    
    for i, k in enumerate(resp_keys):
        survey_data[k] = user_responses[i]
        
    return survey_data


def process_ui_data(data, user):
    '''process the ui data from the session_data into a DF of important timestamps and a dictionary of survey data'''
    
    SURVEY_NAMES = ['LabStartSurvey', 
                    'LabMid1Survey', 
                    'LabMidActivitySurvey', 
                    'LabMid2Survey', 
                    'LabFinalSurvey',
                    'HomeStartSurvey', 
                    'HomeMidSurvey', 
                    'HomeMidActivitySurvey',  
                    'HomeFinalSurvey']          
                    
    # Define lists for two dataframes
    timestamps = []
    surveys = {}
    current_survey = 'UNKNOWN'
    # Traverse through data
    for row in data:
        if row[2] in ['START_TEST', 'STOP_TEST', 'START_TRANSITION', 'FINISHED_TRANSITION', 'NOTICED']:
            timestamps.append([pd.to_datetime(row[0])-datetime.timedelta(hours=TIMEZONE_OFFSET), row[2]])
        elif row[2] == 'SURVEY':
            if 'empaticaStartTime' == row[3]:
                timestamps.append([pd.to_datetime(row[4]+row[5]), 'EMPATICA_START_TIME'])
            elif 'empaticaEndTime' == row[3]:
                timestamps.append([pd.to_datetime(row[4]+row[5]), 'EMPATICA_END_TIME'])
            elif 'recordingStartTime' == row[3]:
                timestamps.append([pd.to_datetime(row[4]+row[5]), 'RECORDING_START_TIME'])
            elif row[4] in SURVEY_NAMES:
                timestamps.append([pd.to_datetime(int(row[3]), unit='ms')-datetime.timedelta(hours=TIMEZONE_OFFSET), 'START_SURVEY'])
                current_survey = row[4]
                surveys[current_survey]={}
            else:
                try:
                    surveys[current_survey][row[3]] = (row[4] if len(row)==5 else row[4:])
                except:
                    print(f'GOT unknown survey: {row}')
    
    # Create dataframes
    df_times = pd.DataFrame(timestamps, columns=['timestamp', 'event'])
    df_times['timestamp'] = pd.to_datetime(df_times['timestamp'], utc=True)
    df_times['timestamp'] = df_times['timestamp'].dt.tz_localize(None)
    df_times.set_index('timestamp', inplace=True)

    for s in surveys:
        print('\t>> processing ' + s)
        surveys[s] = codeBIT(surveys[s])
        surveys[s] = codeFlowDesc(surveys[s])
        print(surveys[s])
        surveys[s] = transformFlowPathPlot(surveys[s], folder, 'P' + str(user) + s)
        
    return df_times, surveys


def codeBIT(survey_data):
    BIT_codes = {
     'BIT0':'Support', 
     'BIT1':'Belonging',
     'BIT2':'Flow',
     'BIT3':'Accomplishment',
     'BIT4':'Self-efficacy',
     'BIT5':'Self-worth',
     'BIT6':'Meaning and Purpose',
     'BIT7':'Optimism',
     'BIT8':'Life Satisfaction',
     'BIT9':'Positive Affect'
    }
    
    all_items=[]
    strengths = []
    weaknesses = []
    
    try:
        for k in BIT_codes.keys():
            score = (int(survey_data[k]) + 1) * 5 / 7
            all_items.append(score)
            if score > 4.2:
                strengths.append(BIT_codes[k])
            elif score < 3.3:
                weaknesses.append(BIT_codes[k])

        BIT_mean = np.mean(all_items)
        BIT_std = np.std(all_items)

        print(f'BIT CODING:\t{BIT_mean:.2f} with std {BIT_std:.2f}', end='')

        if BIT_mean > 4.2:
                print(' -- DESIGNATED "STRONG"')
        elif BIT_std < 3.3:
                print(' -- DESIGNATED "AT RISK"')
                
        print(f'\t\t{len(strengths)} BIT STRENGTHS:\n\t\t {strengths}')
        print(f'\t\t{len(weaknesses)} BIT WEAKNESSES:\n\t\t {weaknesses}')
        
        survey_data['BIT_score'] = BIT_mean
        survey_data['BIT_std'] = BIT_std
        survey_data['BIT_strengths'] = strengths
        survey_data['BIT_weaknesses'] = weaknesses
        
    except:
        print('No BIT Detected')
    
    return survey_data


def codeFlowDesc(survey_data):
    ''' we need all data across all sessions in order to properly z-score flow data.
    instead of doing anything, we'll just process the one bit of data we need to
    preprocess, which is the flowDesc.
    '''
    shallow_flow = ['flowDescA','flowDescB','flowDescC']
    deep_flow = ['flowDescD','flowDescE','flowDescF']
    
    s_score = 0
    d_score = 0
    
    try:
        for k in shallow_flow:
            if survey_data[k] == 'true':
                s_score += 1
        for k in deep_flow:
            if survey_data[k] == 'true':
                d_score += 1
        
        print(f'FLOW DESC:\tshallow: {s_score}\tdeep: {d_score}')
        survey_data['shallowFQ'] = s_score
        survey_data['deepFQ'] = d_score
            
    except:
        print('No Flow Desc Detected')

    return survey_data


def transformFlowPathPlot(survey_data, folder, filename):
    
    try:
        xy = survey_data['flowPath']
        canvassize = survey_data['flowCanvasSize']
    except:
        print('No flow path data')
        return survey_data
    
    if xy=='':
        print('No flow path data')
        return survey_data
    
    xy_float = [float(i) for i in xy]

    # Create list of dictionaries representing coordinates
    coordinates = [{'x': xy_float[i], 'y': xy_float[i + 1]} for i in range(0, len(xy_float), 2)]

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(coordinates)

    # Define the size of the plot to match your canvas size (converted to inches)
    fig, ax = plt.subplots(figsize=(float(canvassize[0]) / 40, float(canvassize[1]) / 40))

    # Create the scatterplot
    sns.lineplot(data=df, x='x', y='y', lw=5, ax=ax)
    
    # Set title and labels
    plt.title('Flow Drawing', fontsize=12)
    #plt.set_xlabel('Time', fontsize=22)
    #plt.set_ylabel('Value', fontsize=22)
    
    ax.set_xticks([0, 5, float(canvassize[0])-5, float(canvassize[0])])
    ax.set_xticklabels(['','Start of Activity', 'End of Activity',''], fontsize=8)
    ax.set_yticks([0, 2, float(canvassize[1])/2, float(canvassize[1])-2, float(canvassize[1])])
    ax.set_yticklabels(['','Distracted', 'Light Focus', 'Deep Focus',''], fontsize=10)

    # Add x and y labels
    ax.set_xlabel('Time', fontsize=8)
    
    plt.xlim([0, 327.5])
    plt.ylim([0, 100])
    
    # Invert the y axis to match the typical screen/canvas coordinate system
    ax.invert_yaxis()

    # Remove tick marks
    plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, left=False, right=False, labelleft=False)

    survey_data['flowDrawing'] = folder + '/Flow_Drawings/' + filename + '.png'
    plt.savefig(survey_data['flowDrawing'], dpi=300)
    plt.close()
    
    print('SAVED ' + survey_data['flowDrawing'])
    
    return survey_data 


def clean_survey_types(session_data):

    #iterate over sessions
    for i, session in enumerate(session_data):
        #iterate over surveys in this session
        if 'surveys' in session:
            for survey in session['surveys']:
                print(f'Cleaning survey {survey}')
                clean_survey(session['surveys'][survey])

    
def clean_survey(survey): 
    #all out of seven EXCEPT nowEmotion and nowEmoIntensity, which are 5 for SAM
    #all start as zero indexed, move to one indexed
    likert_ints = ['FSS', 'flowQ', 'now', 'timeExp', 'focus', 'effort', 'deepest']        

    #these are one indexed and have a map in the data (except percentFlow/duration/durGuess, which are just mapped directly to an int
    map_ints = ['durFlow', 'durTimeConfidence', 'guessTimeConfidence', 'percentFlow', 'duration', 'durGuess']

    times = ['LastSeenTimeStartSurvey','actualTimeAtDuration','startSurveyTime','timeAtDurGuess','timeAtGuess']

    try:
        d = survey['timeAtGuess'][0]
    except:
        pass
    
    #iterate over each key in this survey
    for k in survey.keys():
        try:
            if k == 'reactionTimesMs':
                survey['reactionTimesMs'] = [int(t) for t in survey['reactionTimesMs']]
            elif k in map_ints:
                survey[k] = int(survey[k])
            elif k in times:
                survey[k] = datetime.datetime.strptime(''.join(survey[k]), '%m/%d/%Y %I:%M:%S %p')
            elif k == 'guessTime':
                try:
                    survey['guessTime'] = datetime.datetime.strptime(d + ' ' + survey['guessTime'], '%m/%d/%Y %I:%M %p')
                except:
                    print('Failed on guessTime; no timeAtGuess date to transform')
            else:
                #iterate over likert_ints, because some are partial names
                for st in likert_ints:
                    #with partial match to k
                    if st in k:
                        #match plus 1
                        survey[k] = int(survey[k])+1
        except:
            print(f'\tfailed on {k} with value {survey[k]}')

def process_surveys(folder, user, session_data):
 
    entrance_survey, exit_survey = {}, {}
    
    print('='*10 + ' Entrance Survey Processing')
    try:
        entrance_survey = get_user_entrance_responses(folder, user)
        print('GOT ENTRANCE SURVEY: ' + str(entrance_survey))
        clean_survey(entrance_survey)
    except Exception as e:
        print(e)
        print('NO ENTRANCE SURVEY FOR THIS PARTICIPANT!!!')
 
    print('='*10 + ' Exit Survey Processing')
    try:
        exit_survey = get_user_exit_responses(folder, user)
        print('GOT EXIT SURVEY: ' + str(exit_survey))
        clean_survey(exit_survey)
    except Exception as e:
        print(e)
        print('NO EXIT SURVEY FOR THIS PARTICIPANT!!!')

        
    for session in session_data:
        try:
            print('='*10, end=' ')
            print(f"Processing session {session['timestamp']} {session['user']} {session['session']} {session['session_type']}")
            df_times, survey_results = process_ui_data(session['ui'], user)
            print(df_times)
            print('GOT UI survey results: ' + str(survey_results))
            del session['ui']
            session['surveys'] = survey_results
            session['timings'] = df_times
        except Exception as e:
            print(e)
            print('FAILED SESSION PROCESSING')
            
    clean_survey_types(session_data)
    
    return session_data, entrance_survey, exit_survey
    return session_data, entrance_survey, exit_survey

# Runs the above scripts to generate our data!!

In [10]:
users_to_process = [9, 15]
folder = 'Data'

for user in users_to_process:
    #grab all sections of Session 1 for that user (glasses CSVs, each separated by task/env')
    session_data = load_session_excel(folder, 'P' + str(user), 'sess1')
    update_sessions_with_E4_data(session_data, folder, user, 1)
    
    #grab all sections of Session 2
    sess2_data = load_session_excel(folder, 'P' + str(user), 'sess2')
    update_sessions_with_E4_data(sess2_data, folder, user, 2)
        
    #combine into one
    session_data.extend(sess2_data)
    
    #process the survey data for each session
    session_data, entrance, exit = process_surveys(folder, user, session_data)
    
    #save to a pickle
    with open(folder + '/P' + str(user) + '_Processed.pickle', 'wb') as handle:
        pickle.dump((session_data, entrance, exit), handle, protocol=pickle.HIGHEST_PROTOCOL)



Processing Data/P9/P9_sess1/042123_103001_p9_LAB_DEMO.csv
got raw data.
	thermal processing...
		dropped 0 rows of data due to duplication.
		dropped 0 rows of data due to weird timestamps.
		-----------------------------
		Duration: 7 minutes and 25.90 seconds
		Error (server timestamp vs tick): 0.01 seconds
		% drift: -0.0033639829558196905
	gyro processing...
		dropped 55 rows of data due to duplication.
		dropped 20 rows of data due to weird timestamps.
		-----------------------------
		Duration: 7 minutes and 25.91 seconds
		Error (server timestamp vs tick): 0.00 seconds
		% drift: 0.00033639150589021526
	accelerometer processing...
		dropped 21 rows of data due to duplication.
		dropped 4 rows of data due to weird timestamps.
		-----------------------------
		Duration: 7 minutes and 25.41 seconds
		Error (server timestamp vs tick): 0.00 seconds
		% drift: 0.0006735359853438569
	quaternion extended kalman filter application...
	blink processing...
		dropped 245 rows of data due 

/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/pympler/asizeof.py:1127: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = _getsizeof(obj, s)
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/pympler/asizeof.py:1127: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = _getsizeof(obj, s)
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/pympler/asizeof.py:1127: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = _getsizeof(obj, s)
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/pympler/asizeof.py:1127: FutureWarning: The default dtype for empty Series will be 'object' instead of 'flo

Failed to add E4 ACC for session HOME_TETRIS
'blinks'
Failed to add E4 IBI for session HOME_TETRIS
'blinks'
Failed to add E4 TEMP for session HOME_TETRIS
'blinks'
Failed to add E4 EDA for session HOME_TETRIS
'blinks'
Failed to add E4 BVP for session HOME_TETRIS
'blinks'
Failed to add E4 HR for session HOME_TETRIS
'blinks'
========== Entrance Survey Processing
BIT CODING:	nan with std nan		0 BIT STRENGTHS:
		 []
		0 BIT WEAKNESSES:
		 []
FLOW DESC:	shallow: 0	deep: 0
list index out of range
NO ENTRANCE SURVEY FOR THIS PARTICIPANT!!!
========== Exit Survey Processing
BIT CODING:	2.50 with std 0.97 -- DESIGNATED "AT RISK"
		0 BIT STRENGTHS:
		 []
		7 BIT WEAKNESSES:
		 ['Belonging', 'Flow', 'Accomplishment', 'Meaning and Purpose', 'Optimism', 'Life Satisfaction', 'Positive Affect']
FLOW DESC:	shallow: 1	deep: 2
GOT EXIT SURVEY: {'BIT_score': 2.4999999999999996, 'BIT_std': 0.9715336077668174, 'BIT_strengths': [], 'BIT_weaknesses': ['Belonging', 'Flow', 'Accomplishment', 'Meaning and Purpos

/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SAVED Data/Flow_Drawings/P9LabMid2Survey.png
                                    event
timestamp                                
2023-04-21 10:48:15.263        START_TEST
2023-04-21 10:59:19.060  START_TRANSITION
2023-04-21 10:59:45.552           NOTICED
2023-04-21 11:19:29.696      START_SURVEY
GOT UI survey results: {'LabMid2Survey': {'startSurveyTime': ['4/21/2023', ' 11:08:43 AM'], 'durGuess': '15', 'timeAtDurGuess': ['4/21/2023', ' 11:09:19 AM'], 'durTimeConfidence': '3', 'guessTime': '10:50 AM', 'timeAtGuess': ['4/21/2023', ' 11:09:37 AM'], 'guessTimeConfidence': '2', 'flowPath': ['18', '48.5', '23', '45.5', '26', '45', '28', '43', '30.5', '42', '34', '40.5', '38.5', '39.5', '43.5', '38.5', '49.5', '38.5', '55', '38.5', '60', '38.5', '65', '39.5', '69', '41.5', '71.5', '44', '74', '46', '76', '48', '77.5', '50.5', '79', '52.5', '80.5', '54.5', '81.5', '57', '83.5', '59', '84', '61', '85', '63', '86', '65.5', '87', '67', '88.5', '68', '89.5', '68.5', '91.5', '68.5', '93.5', '68.5'

/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/pympler/asizeof.py:1127: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = _getsizeof(obj, s)
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/pympler/asizeof.py:1127: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = _getsizeof(obj, s)


The size of the loaded data is 495.84925079345703 MB
========== Entrance Survey Processing
BIT CODING:	nan with std nan		0 BIT STRENGTHS:
		 []
		0 BIT WEAKNESSES:
		 []
FLOW DESC:	shallow: 0	deep: 0
list index out of range
NO ENTRANCE SURVEY FOR THIS PARTICIPANT!!!
========== Exit Survey Processing
BIT CODING:	3.71 with std 0.29 -- DESIGNATED "AT RISK"
		2 BIT STRENGTHS:
		 ['Self-efficacy', 'Self-worth']
		0 BIT WEAKNESSES:
		 []
FLOW DESC:	shallow: 1	deep: 3
GOT EXIT SURVEY: {'BIT_score': 3.714285714285714, 'BIT_std': 0.28571428571428553, 'BIT_strengths': ['Self-efficacy', 'Self-worth'], 'BIT_weaknesses': [], 'shallowFQ': 1, 'deepFQ': 3, 'durGuess': '20', 'durTimeConfidence': '1', 'guessTime': '3:30 PM', 'guessTimeConfidence': '1', 'timeExp': '4', 'focus': '5', 'effort': '2', 'deepest': '5', 'flow': 'Y', 'durFlow': '2', 'durToFlow': '5 min?', 'percentFlow': '30', 'flowDesc': 'There were some moments when it was like I went in, then out, then in again.  Then out.', 'flowQ1': '4', 'fl

/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/davidramsay/.virtualenvs/pyro/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SAVED Data/Flow_Drawings/P15LabFinalSurvey.png
                                     event
timestamp                                 
2023-04-14 14:55:48.577         START_TEST
2023-04-14 15:09:25.416   START_TRANSITION
2023-04-14 15:10:02.437            NOTICED
2023-04-14 15:13:43.065          STOP_TEST
2023-04-14 15:13:43.132          STOP_TEST
2023-04-14 15:14:13.172       START_SURVEY
2023-04-14 15:13:27.000  EMPATICA_END_TIME
GOT UI survey results: {'LabFinalSurvey': {'startSurveyTime': ['4/14/2023', ' 3:10:04 PM'], 'durGuess': '20', 'timeAtDurGuess': ['4/14/2023', ' 3:10:07 PM'], 'durTimeConfidence': '2', 'guessTime': '3:30 PM', 'timeAtGuess': ['4/14/2023', ' 3:10:13 PM'], 'guessTimeConfidence': '1', 'flowPath': ['2', '7.5', '7.5', '1.5', '7.5', '-5', '7.5', '-13', '6', '-28', '3', '-44.5', '0', '-60.5', '-3', '-72', '-6', '-84', '-9', '-93.5', '-11', '-100.5', '-13.5', '-109', '-15.5', '-116', '-17', '-125', '-18.5', '-136.5', '-20', '-148.5', '-21', '-158.5'], 'flowCanvasSize': 

-----

-----

-----

# FIX FLIPPED FLOW DATA

A couple of participants mentioned they flipped the y-axis when drawing the flow data in their minds-- i.e. P15 on session 1.

This section will (1) print possible options to fix in the first cell, and (2) if the data is passed, it will invert the y-axis, save a new image, and replace the flowData in the pickle.


In [ ]:
# NEED TO RUN THE FOLLOWING FOR user 15, i=6 surveyname='HomeMidSurvey'

In [11]:
def fixFlowData(session_data, i, surveyname, folder, user):
    '''now we're going to replace the session_data[i]['surveys'][surveyname]['flowPath'] with an inverted flowpath,
    remake the call to transformFlowPathPlot to generate/save the image, and save over the session_data for this user.'''

    #grab the flow path
    flowPath = session_data[i]['surveys'][surveyname]['flowPath']
    canvassize_y = int(session_data[i]['surveys'][surveyname]['flowCanvasSize'][1])
    
    #invert the y-axis
    flowPath[1::2] = [str(canvassize_y-float(i)) for i in flowPath[1::2]]  # invert the flowPath y-axis

    #resave the data
    session_data[i]['surveys'][surveyname]['flowPath'] = flowPath
    
    #save a new flow plot
    session_data[i]['surveys'][surveyname] = transformFlowPathPlot(session_data[i]['surveys'][surveyname], folder, 'P' + str(user) + surveyname)
                                           
    return session_data

In [12]:
#Select the User we need to fix and the Path
folder = 'Data'
user = 15


#let's get a list of options to redo

with open(folder + '/P' + str(user) + '_Processed.pickle', 'rb') as handle:
    session_data, entrance, exit = pickle.load(handle)
    
for i, s in enumerate(session_data):
    for survey in s['surveys']:
        if 'flowPath' in s['surveys'][survey]:
            if len(s['surveys'][survey]['flowPath']):
                print(f'flowPath found in {i}: {s["session_type"]} - {survey}')
                print(f'\t>> call function with ({i}, {survey}')            

flowPath found in 4: FLOW_CONTD - LabFinalSurvey
	>> call function with (4, LabFinalSurvey
flowPath found in 6: TETRIS_CONTD - HomeMidSurvey
	>> call function with (6, HomeMidSurvey
flowPath found in 8: FLOW_CONTD - HomeFinalSurvey
	>> call function with (8, HomeFinalSurvey


In [13]:
#redo and resave; make sure we're still using the correct folder/user from above!

folder = folder
user = user
i = 6
surveyname = 'HomeMidSurvey'

#open it
with open(folder + '/P' + str(user) + '_Processed.pickle', 'rb') as handle:
    session_data, entrance, exit = pickle.load(handle)
    
#fix the session_data to invert the y-axis in flowPath and redraw/resave the drawing
session_data = fixFlowData(session_data, i, surveyname, folder, user)

#resave it
with open(folder + '/P' + str(user) + '_Processed.pickle', 'wb') as handle:
        pickle.dump((session_data, entrance, exit), handle, protocol=pickle.HIGHEST_PROTOCOL)

SAVED Data/Flow_Drawings/P15HomeMidSurvey.png
